### Import Dependencies


In [3]:
!pip install pyspark

In [4]:
!pip install pmdarima

In [5]:
# Data wrangling
import pandas as pd 

# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns 

# Date wrangling
import datetime

# Math operations
import numpy as np

# Random sampling
import random

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

# Keras API 
from tensorflow import keras
from tensorflow.keras import layers

# Deep learning 
from keras.models import Input, Model, Sequential
from keras.layers import Dense, Dropout, LSTM, Concatenate, SimpleRNN, Masking, Flatten
from keras import losses
from keras.callbacks import EarlyStopping
from keras.initializers import RandomNormal

# Time Series
import pmdarima as pm
from pmdarima import model_selection
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

# pyspark related
# pip install pyspark
import pyspark
from pyspark.sql.functions import split, explode, to_timestamp, from_unixtime, from_utc_timestamp
from pyspark import SparkConf, SparkContext,SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,ArrayType,StringType,DoubleType
from pyspark.sql.functions import *
from pyspark.sql import Window
# from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
#from ts.flint import FlintContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


### Get Cleaned Final Data

In [6]:
%%bash
if [ ! -d '/content/data' ]; then
  git clone https://gitlab.com/Dimu_1020/big_data_final/ '/content/data'
  cd '/content/data'
else
  echo "Dataset already downloaded in '/content/data'"
fi

Dataset already downloaded in '/content/data'


In [7]:
data = pd.read_pickle("/content/data/pd_final.pkl")
data

,temp,day_cos,day_sin,month_sin,month_cos,pressure,wind_speed,weather_main,weather_id
0,280.34,1.000000,0.000000,-0.004301,0.999991,1030.0,6.70,foggy,3
1,280.34,1.000000,0.000000,-0.004301,0.999991,1030.0,6.70,rainy,4
2,280.34,1.000000,0.000000,-0.004301,0.999991,1030.0,6.70,rainy,4
3,280.34,0.965926,0.258819,-0.003584,0.999994,1030.0,6.70,foggy,3
4,280.34,0.965926,0.258819,-0.003584,0.999994,1030.0,6.70,rainy,4
...,...,...,...,...,...,...,...,...,...
416544,289.09,0.258819,-0.965926,-0.340673,0.940182,1004.0,0.45,cloudy,1
416545,289.64,0.500000,-0.866025,-0.339999,0.940426,1003.0,0.45,cloudy,1
416546,291.16,0.707107,-0.707107,-0.339325,0.940669,1003.0,0.89,cloudy,1
416547,291.23,0.866025,-0.500000,-0.338650,0.940912,1003.0,0.89,rainy,4


In [8]:
data_arima = pd.read_pickle("/content/data/pd_arima.pkl")
data_arima

,dt,temp
0,1979-01-01 00:00:00,280.34
1,1979-01-01 00:00:00,280.34
2,1979-01-01 00:00:00,280.34
3,1979-01-01 01:00:00,280.34
4,1979-01-01 01:00:00,280.34
...,...,...
416544,2021-12-11 19:00:00,289.09
416545,2021-12-11 20:00:00,289.64
416546,2021-12-11 21:00:00,291.16
416547,2021-12-11 22:00:00,291.23


## Models

#### Set hyper-parameters

In [9]:
# Share of obs in testing 
train_test_split_share = 0.1

# Number of lags (hours back) to use for models
lag = 48

# Steps ahead to forecast 
n_ahead = 1

# select features
features_to_use = ['temp', 'day_cos', 'day_sin', 'month_sin', 'month_cos', 'pressure', 'wind_speed']
keep = ['weather_id']

# features for two models
feature_temp = features_to_use
feature_id = features_to_use + keep

#### Scaler
Apply Standardscaler to all related features

In [10]:
data[feature_temp]

,temp,day_cos,day_sin,month_sin,month_cos,pressure,wind_speed
0,280.34,1.000000,0.000000,-0.004301,0.999991,1030.0,6.70
1,280.34,1.000000,0.000000,-0.004301,0.999991,1030.0,6.70
2,280.34,1.000000,0.000000,-0.004301,0.999991,1030.0,6.70
3,280.34,0.965926,0.258819,-0.003584,0.999994,1030.0,6.70
4,280.34,0.965926,0.258819,-0.003584,0.999994,1030.0,6.70
...,...,...,...,...,...,...,...
416544,289.09,0.258819,-0.965926,-0.340673,0.940182,1004.0,0.45
416545,289.64,0.500000,-0.866025,-0.339999,0.940426,1003.0,0.45
416546,291.16,0.707107,-0.707107,-0.339325,0.940669,1003.0,0.89
416547,291.23,0.866025,-0.500000,-0.338650,0.940912,1003.0,0.89


In [11]:
scaler = StandardScaler()
np_final_temp = scaler.fit_transform(data[feature_temp])
np_final_id = np.append(np_final_temp, data['weather_id'].values.reshape(-1,1).astype('int'), axis=1)

#### Train Test Split

convert to time series dataset for train_test_split

In [12]:
def create_X_Y(ts: np.array, _lag=48, n_ahead=1, target_index=0) -> tuple:
    """
    - ts: A time series dataframe in series form
    - lag: Number of lags (hours back) to use for models
    - n_ahead: Steps ahead to forecast 
    Using <lag> number of data: x_{i}, x_{i+1}, ...x_{i+lag-1} to predict <n_ahead>
    number of data x_{i+lag}, ..., x_{i+lag+n_ahead-1}
    Thus, X = x_{i}, x_{i+1}, ...x_{i+lag-1}
        , Y = x_{i+lag}, ..., x_{i+lag+n_ahead-1}
    """
    # Extract # of features and # of observations
    n_obs = ts.shape[0]
    n_features = ts.shape[1]
    
    # Creating placeholder lists
    X, Y = [], []

    # if we don't have enough we don't have enough obs to predict Y
    if n_obs - _lag <= 0:
        X.append(ts) # no label, only x
    else:
        for i in range(n_obs - _lag - n_ahead):
            Y.append(ts[(i + _lag):(i + _lag + n_ahead), target_index])
            X.append(ts[i:(i + _lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an RNN input shape 
    X = np.reshape(X, (X.shape[0], _lag, n_features))

    return X, Y

In [13]:
def my_train_test_split(ts: np.array, _lag=48, n_ahead=1, target_index=0, if_binary=True) -> tuple:
    X, Y = create_X_Y(ts, 48, n_ahead=n_ahead, target_index=target_index)
    X_train, X_test_, y_train, y_test_ = train_test_split(X, Y, random_state=0)
    def _to_binary(y):
        # encode increasing as 1 and decrease as 0
        y_binary = np.where(np.diff(y, axis = 0) > 0, 1, 0)
        return y_binary
    if if_binary:
        y_train = _to_binary(y_train)
        y_test_ = _to_binary(y_test_)
        return X_train[:-1], X_test_[:-1], y_train, y_test_
    return X_train, X_test_, y_train, y_test_

### Transformer

In [26]:
# temperature model, head = 1
X_train_temp, X_test_temp, y_train_temp, y_test_temp = my_train_test_split(np_final_temp, 48, 1, 0, 1)

# temperature model, head = 7
X_train_temp_7, X_test_temp_7, y_train_temp_7, y_test_temp_7 = my_train_test_split(np_final_temp, 48, 7, 0, 1)

# id model, head = 1
X_train_id, X_test_id, y_train_id, y_test_id = my_train_test_split(np_final_id, 48, 1, 7, 0)

# id model, head = 7
X_train_id_7, X_test_id_7, y_train_id_7, y_test_id_7 = my_train_test_split(np_final_id, 48, 7, 7, 0)

In [27]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(7, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

#### 1. temperature, head = 1

In [ ]:
#####################
X_train = X_train_temp
y_train = y_train_temp
#####################

input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=20,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25
)

model.compile(
    loss=losses.MeanAbsoluteError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    # metrics=["sparse_categorical_accuracy"],
)
# model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=256,
    callbacks=callbacks,
)


In [ ]:
model.save("Transformer_head1.h5")
from google.colab import files
files.download("/content/Transformer_head1.h5")

In [ ]:
model.evaluate(X_test_temp, y_test_temp)

#### 2. temperature, head = 7

In [28]:
#####################
X_train = X_train_temp_7
y_train = y_train_temp_7
#####################

input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=20,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25
)

model.compile(
    loss=losses.MeanAbsoluteError(), 
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[keras.metrics.CategoricalAccuracy()]
)
# model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=256,
    callbacks=callbacks,
)


Epoch 1/5
977/977 [==============================] - 563s 572ms/step - loss: 0.2910 - categorical_accuracy: 0.1142 - val_loss: 0.2626 - val_categorical_accuracy: 0.1730
Epoch 2/5
977/977 [==============================] - 560s 573ms/step - loss: 0.2649 - categorical_accuracy: 0.2299 - val_loss: 0.2620 - val_categorical_accuracy: 0.3254
Epoch 3/5
977/977 [==============================] - 556s 569ms/step - loss: 0.2644 - categorical_accuracy: 0.3031 - val_loss: 0.2618 - val_categorical_accuracy: 0.3819
Epoch 4/5
977/977 [==============================] - 552s 565ms/step - loss: 0.2641 - categorical_accuracy: 0.3609 - val_loss: 0.2609 - val_categorical_accuracy: 0.4154
Epoch 5/5
977/977 [==============================] - 551s 564ms/step - loss: 0.2637 - categorical_accuracy: 0.3691 - val_loss: 0.2604 - val_categorical_accuracy: 0.4166


In [ ]:
model.save("Transformer_head7.h5")
from google.colab import files
files.download("/content/Transformer_head7.h5")

In [ ]:
model.evaluate(X_test_temp_7, y_test_temp_7)

#### 3. id, head = 1

In [ ]:
#####################
X_train = X_train_id
y_train = y_train_id
#####################

input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=20,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25
)

model.compile(
    loss=losses.MeanAbsoluteError(), 
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
     metrics=[keras.metrics.MeanAbsoluteError()]
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=256,
    # callbacks=callbacks,
)


In [ ]:
model.save("Transformer_head1_id.h5")
from google.colab import files
files.download("/content/Transformer_head1_id.h5")

#### id, head = 7

In [ ]:
#####################
X_train = X_train_id_7
y_train = y_train_id_7
#####################

input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=20,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25
)

model.compile(
    loss=losses.MeanAbsoluteError(), 
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
     metrics=[keras.metrics.MeanAbsoluteError()]
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=256,
    # callbacks=callbacks,
)


In [ ]:
model.save("Transformer_head7_id.h5")
from google.colab import files
files.download("/content/Transformer_head7_id.h5")

In [51]:
Transformer_head1 = keras.models.load_model("/content/Transformer_head1.h5")
Transformer_head1_id = keras.models.load_model("/content/Transformer_head1_id.h5")
Transformer_head7 = keras.models.load_model("/content/Transformer_head7.h5")
Transformer_head7_id = keras.models.load_model("/content/Transformer_head7_id.h5")

In [55]:
Transformer_head1.evaluate(X_test_temp, y_test_temp)
Transformer_head7.evaluate(X_test_temp_7, y_test_temp_7)


3254/3254 [==============================] - 92s 28ms/step - loss: 0.2629 - categorical_accuracy: 0.4223


[0.2629221975803375, 0.422279417514801]

In [60]:
Transformer_head1_id.evaluate(X_test_id, y_test_id)
Transformer_head7_id.evaluate(X_test_id_7, y_test_id_7)

3254/3254 [==============================] - 91s 28ms/step - loss: 1.2511 - mean_absolute_error: 1.2511


[1.251085877418518, 1.251085877418518]

### LSTM

In [14]:
# temperature model, head = 1
X_train_temp, X_test_temp, y_train_temp, y_test_temp = my_train_test_split(np_final_temp, 48, 1, 0, 0)

# temperature model, head = 7
X_train_temp_7, X_test_temp_7, y_train_temp_7, y_test_temp_7 = my_train_test_split(np_final_temp, 48, 7, 0, 0)

# id model, head = 1
X_train_id, X_test_id, y_train_id, y_test_id = my_train_test_split(np_final_id, 48, 1, 7, 0)

# id model, head = 7
X_train_id_7, X_test_id_7, y_train_id_7, y_test_id_7 = my_train_test_split(np_final_id, 48, 7, 7, 0)

In [ ]:
class NNMultistepModel():
    
    def __init__(
        self, 
        X, 
        Y, 
        n_outputs,
        n_lag,
        n_ft,
        n_layer,
        batch,
        epochs, 
        lr,
        Xval=None,
        Yval=None,
        mask_value=-999.0,
        min_delta=0.001,
        patience=5
    ):
        lstm_input = Input(shape=(n_lag, n_ft))

        # Series signal 
        lstm_layer = LSTM(n_layer, activation='relu')(lstm_input)

        x = Dense(n_outputs)(lstm_layer)
        
        self.model = Model(inputs=lstm_input, outputs=x)
        self.batch = batch 
        self.epochs = epochs
        self.n_layer=n_layer
        self.lr = lr 
        self.Xval = Xval
        self.Yval = Yval
        self.X = X
        self.Y = Y
        self.mask_value = mask_value
        self.min_delta = min_delta
        self.patience = patience

    def trainCallback(self):
        return EarlyStopping(monitor='loss', patience=self.patience, min_delta=self.min_delta)  

    def train(self):
        # Getting the untrained model 
        empty_model = self.model
        
        # Initiating the optimizer
        optimizer = keras.optimizers.Adam(learning_rate=self.lr)

        # Compiling the model
        empty_model.compile(optimizer=optimizer,
                            loss=losses.MeanAbsoluteError(), 
                            # metrics=[keras.metrics.sparse_categorical_accuracy()]
                            )

        if (self.Xval is not None) & (self.Yval is not None):
            history = empty_model.fit(
                self.X, 
                self.Y, 
                epochs=self.epochs, 
                batch_size=self.batch, 
                validation_data=(self.Xval, self.Yval), 
                shuffle=False,
                callbacks=[self.trainCallback()]
            )
        else:
            history = empty_model.fit(
                self.X, 
                self.Y, 
                epochs=self.epochs, 
                batch_size=self.batch,
                shuffle=False,
                callbacks=[self.trainCallback()]
            )
        
        # Saving to original model attribute in the class
        self.model = empty_model
        
        # Returning the training history
        return history
    
    def predict(self, X):
        return self.model.predict(X)
    
    def evaluate(self, X, y):
        return self.model.evaluate(X, y)

#### temperature, head = 1

In [ ]:
#####################
X_train = X_train_temp[: 218661]
X_val = X_train_temp[218661:]
y_train = y_train_temp[: 218661]
y_val = y_train_temp[218661:]
#####################
model = NNMultistepModel(
    X=X_train,
    Y=y_train,
    n_outputs=n_ahead,
    n_lag=lag,
    n_ft=X_train.shape[2], 
    n_layer=10,
    batch=512,
    epochs=20, 
    lr=0.001,
    Xval=X_val,
    Yval=y_val,
)
# model.model.summary()
history = model.train()

Epoch 1/20
428/428 [==============================] - 105s 38ms/step - loss: 0.2311 - val_loss: 0.0983
Epoch 2/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0717 - val_loss: 0.0606
Epoch 3/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0589 - val_loss: 0.0582
Epoch 4/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0574 - val_loss: 0.0571
Epoch 5/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0567 - val_loss: 0.0568
Epoch 6/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0562 - val_loss: 0.0565
Epoch 7/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0559 - val_loss: 0.0562
Epoch 8/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0557 - val_loss: 0.0560
Epoch 9/20
428/428 [==============================] - 16s 37ms/step - loss: 0.0555 - val_loss: 0.0556
Epoch 10/20
428/428 [==============================] - 16s 37ms/step - loss: 0.05

In [1]:
model.evaluate(X_test_temp, y_test_temp)

NameError: ignored

In [ ]:
model.model.save("LSTM_head1.h5")
files.download("/content/LSTM_head1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### temperature, head = 7

In [ ]:
#####################
X_train = X_train_temp_7[: 218661]
X_val = X_train_temp_7[218661:]
y_train = y_train_temp_7[: 218661]
y_val = y_train_temp_7[218661:]
#####################
model = NNMultistepModel(
    X=X_train,
    Y=y_train,
    n_outputs=n_ahead,
    n_lag=lag,
    n_ft=X_train.shape[2], 
    n_layer=10,
    batch=512,
    epochs=20, 
    lr=0.001,
    Xval=X_val,
    Yval=y_val,
)
# model.model.summary()
history = model.train()

Epoch 1/20
428/428 [==============================] - 32s 68ms/step - loss: 0.2508 - val_loss: 0.1583
Epoch 2/20
428/428 [==============================] - 28s 66ms/step - loss: 0.1441 - val_loss: 0.1371
Epoch 3/20
428/428 [==============================] - 30s 71ms/step - loss: 0.1358 - val_loss: 0.1347
Epoch 4/20
428/428 [==============================] - 16s 37ms/step - loss: 0.1343 - val_loss: 0.1339
Epoch 5/20
428/428 [==============================] - 16s 38ms/step - loss: 0.1335 - val_loss: 0.1328
Epoch 6/20
428/428 [==============================] - 17s 40ms/step - loss: 0.1328 - val_loss: 0.1321
Epoch 7/20
428/428 [==============================] - 17s 39ms/step - loss: 0.1323 - val_loss: 0.1316
Epoch 8/20
428/428 [==============================] - 16s 37ms/step - loss: 0.1319 - val_loss: 0.1312
Epoch 9/20
428/428 [==============================] - 17s 40ms/step - loss: 0.1315 - val_loss: 0.1308
Epoch 10/20
428/428 [==============================] - 16s 37ms/step - loss: 0.131

In [ ]:
model.evaluate(X_test_temp_7, y_test_temp_7)

In [ ]:
model.model.save("LSTM_head7.h5")
files.download("/content/LSTM_head7.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### id, head = 1

In [ ]:
#####################
X_train = X_train_id[: 218661]
X_val = X_train_id[218661:]
y_train = y_train_id[: 218661]
y_val = y_train_id[218661:]
#####################
model = NNMultistepModel(
    X=X_train,
    Y=y_train,
    n_outputs=n_ahead,
    n_lag=lag,
    n_ft=X_train.shape[2], 
    n_layer=10,
    batch=512,
    epochs=20, 
    lr=0.001,
    Xval=X_val,
    Yval=y_val,
)
# model.model.summary()
history = model.train()

Epoch 1/20
428/428 [==============================] - 34s 69ms/step - loss: 0.6667 - val_loss: 0.4556
Epoch 2/20
428/428 [==============================] - 24s 57ms/step - loss: 0.4519 - val_loss: 0.4436
Epoch 3/20
428/428 [==============================] - 19s 43ms/step - loss: 0.4458 - val_loss: 0.4410
Epoch 4/20
428/428 [==============================] - 17s 39ms/step - loss: 0.4430 - val_loss: 0.4388
Epoch 5/20
428/428 [==============================] - 16s 37ms/step - loss: 0.4408 - val_loss: 0.4396
Epoch 6/20
428/428 [==============================] - 16s 37ms/step - loss: 0.4390 - val_loss: 0.4348
Epoch 7/20
428/428 [==============================] - 16s 37ms/step - loss: 0.4370 - val_loss: 0.4333
Epoch 8/20
428/428 [==============================] - 16s 37ms/step - loss: 0.4333 - val_loss: 0.4298
Epoch 9/20
428/428 [==============================] - 16s 37ms/step - loss: 0.4292 - val_loss: 0.4262
Epoch 10/20
428/428 [==============================] - 16s 38ms/step - loss: 0.426

In [ ]:
model.evaluate(X_test_id, y_test_id)

3254/3254 [==============================] - 16s 5ms/step - loss: 0.4148


0.4148290157318115

In [49]:
model.model.save("LSTM_id_head1.h5")
files.download("/content/LSTM_id_head1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### id, head = 7

In [ ]:
#####################
X_train = X_train_id_7[: 218661]
X_val = X_train_id_7[218661:]
y_train = y_train_id_7[: 218661]
y_val = y_train_id_7[218661:]
#####################
model = NNMultistepModel(
    X=X_train,
    Y=y_train,
    n_outputs=n_ahead,
    n_lag=lag,
    n_ft=X_train.shape[2], 
    n_layer=10,
    batch=512,
    epochs=20, 
    lr=0.001,
    Xval=X_val,
    Yval=y_val,
)
# model.model.summary()
history = model.train()

Epoch 1/20
428/428 [==============================] - 17s 38ms/step - loss: 0.8199 - val_loss: 0.6909
Epoch 2/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6825 - val_loss: 0.6797
Epoch 3/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6767 - val_loss: 0.6769
Epoch 4/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6743 - val_loss: 0.6744
Epoch 5/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6724 - val_loss: 0.6727
Epoch 6/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6705 - val_loss: 0.6711
Epoch 7/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6680 - val_loss: 0.6691
Epoch 8/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6658 - val_loss: 0.6665
Epoch 9/20
428/428 [==============================] - 16s 37ms/step - loss: 0.6642 - val_loss: 0.6653
Epoch 10/20
428/428 [==============================] - 16s 37ms/step - loss: 0.663

In [ ]:
model.evaluate(X_test_id_7, y_test_id_7)

3254/3254 [==============================] - 16s 5ms/step - loss: 0.6538


0.6538137793540955

In [50]:
model.model.save("LSTM_id_head7.h5")
files.download("/content/LSTM_id_head7.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
LSTM_head1 = keras.models.load_model("/content/LSTM_head1.h5")
LSTM_head1_id = keras.models.load_model("/content/LSTM_id_head1.h5")
LSTM_head7 = keras.models.load_model("/content/LSTM_head7.h5")
LSTM_head7_id = keras.models.load_model("/content/LSTM_id_head7.h5")

In [18]:
LSTM_head1.evaluate(X_test_temp, y_test_temp)
LSTM_head7.evaluate(X_test_temp_7, y_test_temp_7)


3254/3254 [==============================] - 20s 6ms/step - loss: 0.1298


0.12984268367290497

In [ ]:
LSTM_head1_id.evaluate(X_test_id, y_test_id)
LSTM_head7_id.evaluate(X_test_id_7, y_test_id_7)